In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from lsst.daf.butler import Butler
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
butler = Butler('/repo/LSSTComCam', collections=["LSSTComCam/raw/all","LSSTComCam/calib"])
client = EfdClient('summit_efd')
data_dir = '/home/cslage/DATA'
instrument = "LSSTComCam"

In [ ]:
expIds = []
lenDatas = []
expTimes = []
for expId in range(2024072900178, 2024072900190):
    mData = butler.get('raw.metadata', detector=4, exposure=expId, instrument=instrument)
    print(mData['DATE-END'])
    start = Time(mData['DATE-END'], scale='tai').utc
    end = start + TimeDelta(8.0, format='sec')
    urls = await client.select_time_series('lsst.sal.Electrometer.logevent_largeFileObjectAvailable', \
                                            ['*'],  start, end, index=101)
    url = urls.iloc[0]['url']
    filename= os.path.split(url)[1]
    elec_filen = os.path.join(data_dir, filename)
    os.system(f"curl {url} --output {elec_filen}") 
    hdu = fits.open(f'{elec_filen}')
    data = hdu[1].data
    expIds.append(expId)
    lenDatas.append(len(data))
    expTimes.append(mData['EXPTIME'])


In [ ]:
for i in range(len(expIds)):
    print(f"expId={expIds[i]}, expTime={expTimes[i]}, length of LFA={(lenDatas[i]*0.1):.1f}")